In [2]:
import pandas as pd
import polars as pl
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
!pip install sklearn

In [2]:
#!pip install pyarrow

In [2]:
jan_2023 = pl.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet')
feb_2023 = pl.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')

In [3]:
jan_2023.head()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
i64,datetime[ns],datetime[ns],str,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64
2,2023-01-01 00:26:10,2023-01-01 00:37:11,"""N""",1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,null,1.0,24.18,1.0,1.0,2.75
2,2023-01-01 00:51:03,2023-01-01 00:57:49,"""N""",1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,null,1.0,15.84,1.0,1.0,0.0
2,2023-01-01 00:35:12,2023-01-01 00:41:32,"""N""",1.0,223,179,1.0,0.0,7.2,1.0,0.5,1.94,0.0,null,1.0,11.64,1.0,1.0,0.0
1,2023-01-01 00:13:14,2023-01-01 00:19:03,"""N""",1.0,41,238,1.0,1.3,6.5,0.5,1.5,1.7,0.0,null,1.0,10.2,1.0,1.0,0.0
1,2023-01-01 00:33:04,2023-01-01 00:39:02,"""N""",1.0,41,74,1.0,1.1,6.0,0.5,1.5,0.0,0.0,null,1.0,8.0,1.0,1.0,0.0


In [5]:
#Q1 There are 20 columns in January data and February data

In [4]:
feb_2023.head()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
i32,datetime[ns],datetime[ns],str,i64,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64
1,2023-02-01 00:46:22,2023-02-01 01:05:57,"""N""",1,74,265,1,10.8,42.9,1.0,1.5,0.0,0.0,null,1.0,45.4,2,1,0.0
2,2023-02-01 00:05:09,2023-02-01 00:22:42,"""N""",1,216,196,1,4.76,23.3,1.0,0.5,0.0,0.0,null,1.0,25.8,2,1,0.0
2,2023-02-01 00:03:47,2023-02-01 00:27:30,"""N""",1,7,114,1,6.32,30.3,1.0,0.5,8.89,0.0,null,1.0,44.44,1,1,2.75
2,2023-01-31 23:30:56,2023-01-31 23:51:40,"""N""",1,74,239,1,3.5,16.3,1.0,0.5,0.0,0.0,null,1.0,21.55,2,1,2.75
2,2023-02-01 00:15:05,2023-02-01 00:26:02,"""N""",1,82,223,1,3.14,17.0,1.0,0.5,0.0,0.0,null,1.0,19.5,2,1,0.0


In [ ]:

jan_2023 = jan_2023.with_columns(
(pl.col('lpep_dropoff_datetime')-pl.col('lpep_pickup_datetime')).alias('duration'))

In [6]:
jan_2023.select(pl.std('duration'))

duration
duration[ns]
1h 14m 55s 537873522ns


In [7]:
jan_2023.head()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
i64,datetime[ns],datetime[ns],str,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,duration[ns]
2,2023-01-01 00:26:10,2023-01-01 00:37:11,"""N""",1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,null,1.0,24.18,1.0,1.0,2.75,11m 1s
2,2023-01-01 00:51:03,2023-01-01 00:57:49,"""N""",1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,null,1.0,15.84,1.0,1.0,0.0,6m 46s
2,2023-01-01 00:35:12,2023-01-01 00:41:32,"""N""",1.0,223,179,1.0,0.0,7.2,1.0,0.5,1.94,0.0,null,1.0,11.64,1.0,1.0,0.0,6m 20s
1,2023-01-01 00:13:14,2023-01-01 00:19:03,"""N""",1.0,41,238,1.0,1.3,6.5,0.5,1.5,1.7,0.0,null,1.0,10.2,1.0,1.0,0.0,5m 49s
1,2023-01-01 00:33:04,2023-01-01 00:39:02,"""N""",1.0,41,74,1.0,1.1,6.0,0.5,1.5,0.0,0.0,null,1.0,8.0,1.0,1.0,0.0,5m 58s


In [8]:
import datetime

In [9]:
jan_2023.filter((pl.col('duration')>pl.duration(minutes=1))&(pl.col('duration')<pl.duration(minutes=60)))

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
i64,datetime[ns],datetime[ns],str,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,duration[ns]
2,2023-01-01 00:26:10,2023-01-01 00:37:11,"""N""",1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,null,1.0,24.18,1.0,1.0,2.75,11m 1s
2,2023-01-01 00:51:03,2023-01-01 00:57:49,"""N""",1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,null,1.0,15.84,1.0,1.0,0.0,6m 46s
2,2023-01-01 00:35:12,2023-01-01 00:41:32,"""N""",1.0,223,179,1.0,0.0,7.2,1.0,0.5,1.94,0.0,null,1.0,11.64,1.0,1.0,0.0,6m 20s
1,2023-01-01 00:13:14,2023-01-01 00:19:03,"""N""",1.0,41,238,1.0,1.3,6.5,0.5,1.5,1.7,0.0,null,1.0,10.2,1.0,1.0,0.0,5m 49s
1,2023-01-01 00:33:04,2023-01-01 00:39:02,"""N""",1.0,41,74,1.0,1.1,6.0,0.5,1.5,0.0,0.0,null,1.0,8.0,1.0,1.0,0.0,5m 58s
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2023-01-31 22:29:00,2023-01-31 22:42:00,null,null,49,62,null,4070.82,15.7,0.0,0.0,0.0,0.0,null,1.0,16.7,null,null,null,13m
2,2023-01-31 22:40:00,2023-01-31 22:48:00,null,null,10,205,null,2.14,4.41,0.0,0.0,0.0,0.0,null,1.0,5.41,null,null,null,8m
2,2023-01-31 23:46:00,2023-02-01 00:02:00,null,null,66,37,null,3.44,16.53,0.0,0.0,3.51,0.0,null,1.0,21.04,null,null,null,16m


In [10]:
jan_2023.shape

(68211, 21)

In [11]:
65920/68211

0.96641304188474

In [12]:
filtered_jan_2023 = jan_2023.filter((pl.col('duration')>pl.duration(minutes=1))&(pl.col('duration')<pl.duration(minutes=60)))

In [13]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']


In [14]:
filtered_jan_2023[categorical] = filtered_jan_2023[categorical].with_columns(pl.col('PULocationID').cast(str),pl.col('DOLocationID').cast(str))

In [5]:
 def read_file_to_dataframe(filename):
    
    jan_2023 = jan_2023.with_columns(
    (pl.col('lpep_dropoff_datetime')-pl.col('lpep_pickup_datetime')).alias('duration'))
    jan_2023.filter((pl.col('duration')>pl.duration(minutes=1))&(pl.col('duration')<pl.duration(minutes=60)))
    categorical = ['PULocationID','DOLocationID']
    numerical = ['trip_distance']
    filtered_jan_2023[categorical] = filtered_jan_2023[categorical].with_columns(pl.col('PULocationID').cast(str),pl.col('DOLocationID').cast(str))

In [15]:
train_dicts = filtered_jan_2023[categorical + numerical].to_dicts()

In [16]:
dv = DictVectorizer()

In [17]:
X_train = dv.fit_transform(train_dicts)

In [44]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',
 'DOLocationID=146',
 'DOLocationID=147',

In [67]:
target = 'duration'
y_train = filtered_jan_2023[target].to_numpy()


In [39]:
y_train.dtype

Duration(time_unit='ns')

In [40]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [41]:
y_pred = lr.predict(X_train)

In [36]:
y_pred2=y_pred.astype(np.float64) / 1_000_000_000

In [52]:
y_pred

array([1.09323395e+12, 6.40914166e+11, 6.23852056e+11, ...,
       1.33716528e+12, 1.01704423e+12, 1.19305246e+12])

In [54]:
y_train.dtype

Duration(time_unit='ns')

In [68]:
y_train = (y_train.cast(pl.Int64)).to_numpy()

In [70]:
mean_squared_error(y_train ,y_pred, squared=False)/1_000_000_000

421.79746561495335

In [71]:
421.79746561495335/60

7.029957760249222

In [60]:
#Think I messed up units

In [19]:
def read_file_to_dataframe(filename):
    df = pl.read_parquet(filename)
    
    df = df.with_columns(
    (pl.col('lpep_dropoff_datetime')-pl.col('lpep_pickup_datetime')).alias('duration'))
    
    df = df.filter((pl.col('duration')>pl.duration(minutes=1))&(pl.col('duration')<pl.duration(minutes=60)))
    
#     categorical = ['PULocationID','DOLocationID']
#     numerical = ['trip_distance']
    
#     df[categorical] = df[categorical].with_columns(pl.col('PULocationID').cast(str),pl.col('DOLocationID').cast(str))
    return df

In [20]:
df_train = read_file_to_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet')
df_val = read_file_to_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')

In [18]:
df_train
categorical = ['PULocationID','DOLocationID']
df_train[categorical] = df_train[categorical].with_columns(pl.col('PULocationID').cast(str),pl.col('DOLocationID').cast(str))

In [22]:
categorical = ['PULocationID','DOLocationID']
df_val[categorical] = df_val[categorical].with_columns(pl.col('PULocationID').cast(str),pl.col('DOLocationID').cast(str))

In [30]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dicts()
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dicts()
X_val = dv.fit_transform(val_dicts)

target = 'duration'
y_train = df_train[target].to_numpy()
y_val = df_val[target].to_numpy()


lr = LinearRegression()
lr.fit(X_train,y_train)



y_pred = lr.predict(X_train)
print(y_train.dtype)
print(y_pred.dtype)
y_train_seconds = y_train / np.timedelta64(1, 's')


mean_squared_error(y_train_seconds ,y_pred, squared=False)/1_000_000_000 #gets me to seconds

timedelta64[ns]
float64


828.2188608033437